In [ ]:
%pip install --quiet --upgrade diffusers transformers scipy mediapy accelerate ftfy spacy

In [ ]:
import subprocess

# The xformers package is mandatory to be able to create several 768x768 images.
github_url = "https://github.com/TheLastBen/fast-stable-diffusion"
xformers_wheels = "xformers-0.0.13.dev0-py3-none-any.whl"

# Obtain GPU info

nvidia_output = subprocess.run(['nvidia-smi', '-q'], capture_output=True).stdout

gpu_info = [
    str(line) for line in str(nvidia_output).split('\\n')
    if "Product Name" in line
    ]

print(gpu_info)

# Identify your GPU

gpu_name = gpu_info[0].split()[-1]

assert gpu_name in ['A100', 'K80', 'P100', 'T4', 'V100']

# Install xformers using pre-compiled Python wheels
%pip install -q {github_url}/raw/main/precompiled/{gpu_name}/{xformers_wheels}

In [ ]:
# model_id = "stabilityai/stable-diffusion-2-base"
model_id = "stabilityai/stable-diffusion-2"

In [ ]:
from diffusers import PNDMScheduler, DDIMScheduler, LMSDiscreteScheduler, EulerDiscreteScheduler

# scheduler = PNDMScheduler.from_pretrained(model_id, subfolder="scheduler")
# scheduler = DDIMScheduler.from_pretrained(model_id, subfolder="scheduler")
# scheduler = LMSDiscreteScheduler.from_pretrained(model_id, subfolder="scheduler")
scheduler = EulerDiscreteScheduler.from_pretrained(model_id, subfolder="scheduler")

In [ ]:
import mediapy as media
import torch
from diffusers import StableDiffusionPipeline

device = "cuda"

pipe = StableDiffusionPipeline.from_pretrained(
    model_id,
    scheduler=scheduler,
    torch_dtype=torch.float16,
    revision="fp16",
    )
pipe = pipe.to(device)
pipe.enable_xformers_memory_efficient_attention()

if model_id.endswith('-base'):
  image_length = 512
  num_images = 4
else:
  image_length = 768
  num_images = 1



In [ ]:
prompt = "A pikachu fine dining with a view to the Eiffel Tower"

images = pipe(
    prompt,
    num_images_per_prompt=num_images,
    guidance_scale=9,
    num_inference_steps=25,
    height=image_length,
    width=image_length,
    ).images
    
media.show_images(images)
images[0].save("output.jpg")